In [1]:
!pip install PyGithub
!pip install tqdm

In [2]:
with open("token.txt","r") as fil:
    TOKEN=fil.readline()

In [3]:
import os
import os.path

if os.path.exists("repos.txt"):
    
    with open("repos.txt","r") as fil:
        repos=fil.readlines()

    repos = [x.replace("\n","") for x in repos]

else:
    print("No repo list exists, aborting")
    raise NotImplementedError

In [4]:
import os
import os.path

folder = "repos"

if folder:
    os.chdir(".")

    if os.path.isdir(folder):
        print("Folder already exists")
    else:
        print("Creating folder",folder)
        os.mkdir(folder)


Folder already exists


In [5]:
if os.path.exists("last_worked.txt"):
    
    with open("last_worked.txt","r") as fil:
        last_worked_on=fil.readline()
    position = repos.index(last_worked_on)
    
    last_filename = last_worked_on.replace("/","-")+".csv"
    try:
        os.remove(folder+"/"+last_filename)
    except:
        print("File not found, so not removing it")
    repos = repos[position:]


File not found, so not removing it


In [6]:
from github import Github

g = Github(TOKEN)


In [7]:
import pandas as pd
from tqdm.notebook import tqdm
from github import GithubException


def get_commits_by_repo_name(repo_name):

    with open("last_worked.txt","w") as fil:
        fil.write(repo_name)
    
    authors = []
    dates = []
    empties_count = 0

    repo = g.get_repo(repo_name)
    
    commits = repo.get_commits()
    try:
        
        for commit in tqdm(commits, desc="Commits progress",total=commits.totalCount, position=1, leave=False):
            authors.append(commit.commit.raw_data["author"]["name"])
            dates.append(commit.commit.raw_data["author"]["date"])
    except GithubException as err:
        if err.data["message"] == "Git Repository is empty.":
            print("Found an empty repo.")
            empties_count+=1
        else:
            raise err
    return authors, dates, empties_count

def create_and_save_dataframe(authors, dates, repo_name, folder=""):
    if authors:
        df = pd.DataFrame({"commit_author":authors, "commit_date": dates})
        if folder:
            folder=folder+"/"
        df.to_csv(folder+repo_name.replace("/","-")+".csv")
    return
    


In [ ]:
for repo_name in tqdm(list(repos), desc="Repos progress"):
    authors, dates, empties_count = get_commits_by_repo_name(repo_name)
    create_and_save_dataframe(authors, dates, repo_name, folder)

print("Total nuber of empty repos:", empties_count)


Repos progress:   0%|          | 0/828 [00:00<?, ?it/s]

Commits progress:   0%|          | 0/1219 [00:00<?, ?it/s]